# Práctico 2b: Leaky Integrate and Fire

Este modelo sencillo de neurona captura varios comportamientos de las neuronas reales. Veremos su respuesta ante deferentes estímulos, en particular su patron de respuesta frente al ruido en la entrada.

$$ I(t) = \frac{u(t) - v_{rest}}{R} + C \frac{dv}{dt}$$
$$ \tau_{m} \frac{dv}{dt} = -[v(t)-v_{rest}] + RI(t)$$

## Configuración

Ejecutá las siguientes celdas para configurar el entorno del notebook

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import scipy as sp
import ipywidgets as widgets
from scipy.integrate import odeint
from scipy import stats
from matplotlib import gridspec

### Funciones utilitarias

In [ ]:
def I_inj(t, amp1, dur1, amp2=0, dur2=0, sep=50):
  """Crea una función de corriente inyectada con dos pulsos cuadrados.

  Args:
    t (ndarray): vector 1D de tiempos (ms), estrictamente creciente.
    amp1 (float): amplitud del primer pulso, en uA/cm^2.
    dur1 (float): duración del primer pulso, en ms.
    amp2 (float): amplitud del segundo pulso, en uA/cm^2.
    dur2 (float): duración del segundo pulso, en ms.
    sep (float): separación mínima entre los dos pulsos, en ms.

  Returns:
    function: una función I(t) que devuelve la corriente inyectada I(t).
  """
  tmax = np.max(t)
  inicios = np.linspace(0, tmax, 8)
  inicio1 = inicios[1]
  inicio2 = inicio1 + dur1 + sep
  
  def I(t):
    return (amp1 * (t > inicio1) - amp1 * (t > inicio1 + dur1) +
            amp2 * (t > inicio2) - amp2 * (t > inicio2 + dur2))

  return I

def I_train(t, n, amp, dur, interval, start=50):
  """Crea una función de corriente inyectada con n pulsos cuadrados.

  Args:
    t (ndarray): vector 1D de tiempos (ms), estrictamente creciente.
    n (int): número de pulsos.
    amp (float): amplitud de cada pulso, en uA/cm^2.
    dur (float): duración de cada pulso, en ms.
    interval (float): intervalo entre el inicio de un pulso y el siguiente, en ms.
    start (float): tiempo inicial del primer pulso, en ms.

  Returns:
    function: una función I(t) que devuelve la corriente inyectada I(t).
  """
  # Tiempos de inicio de cada pulso
  inicios = start + np.arange(n) * interval

  def I(t):
    corriente = np.zeros_like(t, dtype=float)
    for inicio in inicios:
      corriente += amp * ((t > inicio) & (t <= inicio + dur))
    return corriente

  return I

def I_inj_ruido(t, amp1=8, amp2=0, media=0, desvio=1):
  tmax = np.max(t)

  if amp2 == 0:
    base = amp1 * (t > 0)
  else:
    base = np.where(t <= tmax/2, amp1, amp2)

  vals = base + np.random.normal(media, desvio, size=t.shape)

  def I(tq):
      return np.interp(tq, t, vals)

  return I

def enmascarar_espigas(t, V, theta):
  """Detecta cruces ascendentes de umbral en el potencial de membrana.

  Args:
    t (ndarray): vector de tiempos (ms).
    V (ndarray): potencial de membrana (mV).

  Returns:
    ndarray: máscara booleana con True en cada cruce ascendente.
  """
  mascara = (V[1:] > theta[1:]) & (V[:-1] <= theta[:-1])
  mascara = np.concatenate(([False], mascara))
  return mascara

def contar_espigas(mascara):
  """Cuenta las espigas a partir de una máscara booleana.

  Args:
    mascara (ndarray): máscara booleana con True en cada cruce ascendente.

  Returns:
    int: número de potenciales de acción detectados.
  """
  return np.sum(mascara)
  
def medir_isis(t, mascara):
  ix, = np.nonzero(mascara)
  return np.diff(t[ix]) if len(ix) > 1 else np.array([])

### Funciones de graficado

In [ ]:
def plot_lif(t, I, V, theta):
  fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(8, 4), gridspec_kw={"height_ratios": [1, 3, 1]}, sharex=True)

  espigas = enmascarar_espigas(t, V, theta)
  axes[0].eventplot(t[espigas])
  axes[0].set_ylabel("Espigas")

  axes[1].plot(t, V, label="Voltaje")
  axes[1].plot(t, theta, 'k--', label="Umbral")
  axes[1].set_ylim(-80, -40)
  axes[1].set_ylabel("Voltaje (mV)")
  axes[1].legend(loc="upper right")

  axes[2].plot(t, I(t), 'r-')
  axes[2].set_ylim(-20, 20)
  axes[2].set_ylabel("Corriente (µA/cm²)")
  axes[2].set_xlabel("Tiempo (ms)")

  plt.tight_layout()
  plt.show()

def plot_isi_hist(isi):
  plt.figure(figsize=(8, 6))
  plt.hist(isi, 20)
  plt.title('Histograma del intervalo entre espigas')
  plt.xlabel('Intervalo entre espigas (ms)')
  plt.ylabel('Frecuencia')
  plt.show()

def plot_I_hist(t, I):
  pulsos = I(t)

  fig, axes = plt.subplots(2, 1, figsize=(8, 6))
  
  # Histograma
  axes[0].hist(pulsos, bins=30)
  axes[0].set_xlabel("Amplitud de la señal de entrada")
  axes[0].set_ylabel("Frecuencia")
  axes[0].set_title("Histograma de la señal de entrada")
  
  # Señal en el tiempo
  axes[1].plot(t, pulsos)
  axes[1].set_xlabel("Tiempo (ms)")
  axes[1].set_ylabel("Corriente")
  
  plt.tight_layout()
  plt.show()

def plot_fi_curve(corrientes, frecuencias):
  plt.scatter(corrientes, frecuencias)
  plt.xlabel("Corriente inyectada (µA/cm²)")
  plt.ylabel("Frecuencia de disparo (Hz)")
  plt.title("Curva F-I (frecuencia vs corriente)")
  plt.show()

def plot_poblacion(t, espigas_pobl, pulsos_pobl):
  """Grafica la actividad poblacional (espigas) y la corriente.

  Args:
    t (ndarray): vector 1D de tiempos (ms), estrictamente creciente.
    espigas_pobl (ndarray): serie temporal de la suma poblacional de espigas
      por ventana temporal.
    pulsos_pobl (ndarray): serie temporal de la corriente (media poblacional),
      por ventana temporal.
  """
  fig, axes = plt.subplots(2, 1, figsize=(8, 6), sharex=True)
  
  axes[0].plot(t, espigas_pobl)
  axes[0].set_ylabel("Número de espigas")
  axes[0].set_title("Suma poblacional de espigas (por ventana)")

  axes[1].plot(t, pulsos_pobl)
  axes[1].set_xlabel("Tiempo (ms)")
  axes[1].set_ylabel("Corriente")
  
  plt.show()

## Definición del modelo

In [ ]:
# Potencial de reposo (mV)
E_L = -65.0

# Constante de tiempo
tau = 5

# Umbral de disparo inicial (mV)
theta_0 = -55.0

# Constante de tiempo del umbral
tau_theta = 20

### Definimos las ecuaciones del modelo

In [ ]:
def integrar_lif(t, I, incr_theta=0):
  dt = t[1] - t[0]     # assume constant step
  pulsos = I(t)                   # input current over time
  V = np.array([E_L])
  theta = np.array([theta_0])

  for i in range(1, len(t)):
    s = V[-1] > theta[-1]
    V = np.append(V, s * E_L + (1 - s) * (V[-1] - dt / tau * ((V[-1] - E_L) - pulsos[i])))
    if s:
      theta = np.append(theta, theta[-1] + s*incr_theta)
    else:
      theta = np.append(theta, theta[-1] - dt / tau_theta * ((theta[-1] - theta_0)))
  return V, theta

## Simulación del modelo

### Corriente externa constante

In [ ]:
@widgets.interact(amp1=(-20, 20, 0.1), dur1=(0, 100), amp2=(-20, 20, 0.1), dur2=(0, 100), sep=(0, 200), incr_theta=(0, 5))
def plot_I_inj(amp1=15, dur1=2, amp2=15, dur2=0, sep=50, incr_theta=0):
  t = np.arange(0, 300, 0.1)
  I = I_inj(t, amp1, dur1, amp2, dur2, sep)
  V, theta = integrar_lif(t, I, incr_theta)
  plot_lif(t, I, V, theta)

### Suma de pulsos de corriente

In [ ]:
@widgets.interact(n=(0, 10), dur=(0, 10), amp=(-20, 20, 0.1), sep=(0, 100), incr_theta=(0, 5))
def plot_I_train(n=5, dur=2, amp=15, sep=3, incr_theta=0):
  t = np.arange(0, 300, 0.1)
  I = I_train(t, n, amp, dur, sep)
  V, theta = integrar_lif(t, I, incr_theta)
  plot_lif(t, I, V, theta)

## Modelando el ruido en la entrada

### Ruido gaussiano en la corriente de entrada

¿Cómo es el patrón de descarga si la corriente de entrada tiene ruido gaussiano?
El parametro <b>media</b> controla la amplitud promedio del ruido, y el <b>desvio</b> controla la variabilidad. Al aumentar la varibilidad, ¿qué pasa con la respuesta de la neurona?

Experimentar:
<ol>
<li> Aumentando la variabilidad.</li>
<li> Aumentando la amplitud (dejando la variabilidad fija, y no muy alta).</li>
<li> Aumentando la amplitud de la entrada de base (RI_ext).</li>

</ol>


In [ ]:
@widgets.interact(amp=(-15, 15, 0.1), media=(-15, 15, 0.1), desvio=(0, 10, 0.1))
def plot_I_inj_ruido(amp=8, media=1, desvio=4):
  np.random.seed(0)  # fijamos la semilla para que el ruido sea reproducible
  t = np.arange(0, 300, 0.1)
  I = I_inj_ruido(t, amp, media=media, desvio=desvio)
  plot_I_hist(t, I)

Corremos y ploteamos el resultado

In [ ]:
@widgets.interact(amp=(-15, 15, 0.1), media=(-15, 15, 0.1), desvio=(0, 10, 0.1), incr_theta=(0, 5))
def plot_I_inj_ruido(amp=8, media=1, desvio=4, incr_theta=0):
  np.random.seed(0)  # fijamos la semilla para que el ruido sea reproducible
  t = np.arange(0, 300, 0.1)
  I = I_inj_ruido(t, amp, media=media, desvio=desvio)
  V, theta = integrar_lif(t, I, incr_theta)
  plot_lif(t, I, V, theta)

### Relación entrada vs. frecuencia de disparo (con ruido)

En ausencia de ruido, la relación entre amplitud de la entrada y frecuencia de disparo es similar a la que vimos en el práctico pasado. ¿Se modifica esa relación ante el ruido?

Probar:
<ol>
<li>La relación sin ruido (amplitud 0, variabilidad 0).</li>
<li>Aumentar la variabilidad (correr varias veces).</li>
<li>Aumentar la amplitud promedio del ruido (corre varias veces)</li>
</ol>

In [ ]:
t = np.arange(0.0, 200.0, 0.1)
duracion = np.max(t) / 1_000  # en segundos

@widgets.interact(amp=(-15, 15, 0.1), media=(-15, 15, 0.1), desvio=(0, 10, 0.1), incr_theta=(0, 5))
def simulate(amp=8, media=1, desvio=4, incr_theta=0):
  np.random.seed(0)  # fijamos la semilla para que el ruido sea reproducible
  
  corrientes = np.arange(0, 12, 0.1)
  frecuencias = np.zeros_like(corrientes)
  
  for i, amp in enumerate(corrientes):
    I = I_inj_ruido(t, amp, media=media, desvio=desvio)
    V, theta = integrar_lif(t, I, incr_theta)
    mascara = enmascarar_espigas(t, V, theta)
    frecuencias[i] = contar_espigas(mascara) / duracion
    
  plot_fi_curve(corrientes, frecuencias)

### Estadística de disparo en la entrada 
Ya vimos la frecuencia promedio en función de la entrada ruidosa. Pero se observa ruido en la salida. La frecuencia parece no ser constante para cada valor de entrada. ¿Qué pasa con los intervalos entre espigas? ¿Cómo se ve la distribución de intervalos entre espigas, y cómo la afecta la amplitud y la variabilidad del ruido en la entrada?
¿Es una distribución realista? (ver figuras 3.5, 3.7 y 3.8 del Trappenberg).

<ul>
<li>Jugar con los parámetros de la entrada, RI_ext y amplitud y variabilidad del ruido</li>
<li>¿Qué pasa si RI_ext es alto y la amplitud es 0 y la variabilidad baja?</li>
<li>Observar la forma del histograma</li>
<li>Mirar el coeficiente de variación (último número que sale debajo de la gráfica)</li>
</ul>


In [ ]:
@widgets.interact_manual(amp=(0,12,0.1), media=(-15, 15, 0.1), desvio=(0, 10, 0.1), incr_theta=(0, 5))
def simulate(amp=4, media=5, desvio=4, incr_theta=0):
  np.random.seed(0)  # fijamos la semilla para que el ruido sea reproducible
  t = np.arange(0.0, 10_000.0, 0.1)
  I = I_inj_ruido(t, amp, media=media, desvio=desvio)
  V, theta = integrar_lif(t, I, incr_theta)

  mascara = enmascarar_espigas(t, V, theta)
  ISIs = medir_isis(t, mascara)
  plot_isi_hist(ISIs)

  print(f"Coeficiente de variación: {stats.variation(ISIs):.4f}")

## Bonificación: Población de neuronas

En esta simulación, cambiamos la entrada externa, en $t = 100$, de una magnitud `amp1` a una magnitud (más alta) `amp2`, para ver como responde una población de neronas a un cambio brusco en la corriente.

In [ ]:
numNeuronas = 1000
t = np.arange(0, 200, 0.5)
duracion = np.max(t) / 1_000  # en segundos

@widgets.interact_manual(amp1=(0,20,0.1), amp2=(0,20,0.1), media=(-15, 15, 0.1), desvio=(0, 10, 0.1), incr_theta=(0, 5))
def simulate(amp1=8, amp2=12, media=5, desvio=4, incr_theta=0):
  espigas = np.zeros([numNeuronas, t.size])
  pulsos = np.zeros([numNeuronas, t.size])
  
  for i in range(numNeuronas):
    I = I_inj_ruido(t, amp1=amp1, amp2=amp2, media=0, desvio=8)
    V, theta = integrar_lif(t, I)
    espigas[i] = enmascarar_espigas(t, V, theta)
    pulsos[i] = I(t)
  
  # Suma poblacional de espigas en cada ventana temporal
  espigas_pobl = espigas.sum(axis=0)

  # Promedio poblacional de la corriente en cada ventana temporal
  pulsos_pobl = pulsos.mean(axis=0)

  plot_poblacion(t, espigas_pobl, pulsos_pobl)

El conteo de disparos sigue el salto en la entrada casi de manera instantánea. La razón es que en cada instante hay un subconjunto de neuronas cerca del umbral. Esas neuronas pueden responder rápido a la entrada, y las otras tienen tiempo de seguirlas con rapidez.